In [2]:
# !pip install selenium
# !pip install webdriver_manager
# !pip install bs4
# !pip install pandas

  Obtaining dependency information for selenium from https://files.pythonhosted.org/packages/3f/fd/c2e7bb547b5b96c7bd536b4a80c4564b7ce5cd38d10095fbba8648996ab9/selenium-4.18.1-py3-none-any.whl.metadata
  Obtaining dependency information for trio~=0.17 from https://files.pythonhosted.org/packages/14/fb/9299cf74953f473a15accfdbe2c15218e766bae8c796f2567c83bae03e98/trio-0.24.0-py3-none-any.whl.metadata
  Obtaining dependency information for trio-websocket~=0.9 from https://files.pythonhosted.org/packages/48/be/a9ae5f50cad5b6f85bd2574c2c923730098530096e170c1ce7452394d7aa/trio_websocket-0.11.1-py3-none-any.whl.metadata
  Obtaining dependency information for outcome from https://files.pythonhosted.org/packages/55/8b/5ab7257531a5d830fc8000c476e63c935488d74609b50f9384a643ec0a62/outcome-1.3.0.post0-py2.py3-none-any.whl.metadata
  Obtaining dependency information for wsproto>=0.14 from https://files.pythonhosted.org/packages/78/58/e860788190eba3bcce367f74d29c4675466ce8dddfba85f7827588416f01/wspro

In [2]:
# conda install selenium

In [74]:
import time
import pandas as pd
import urllib.request
from urllib.parse import quote

import warnings
warnings.filterwarnings('ignore')
from selenium import webdriver  # 동적크롤링
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup

In [75]:
# review_scrapy_data : 전처리가 끝난 visitJeju의 API 전체 CSV
data = pd.read_csv("Data/review_scrapy_data.csv", index_col=False)

In [76]:
data.head()

,title,category,introduction,address,roadaddress,latitude,longitude,phoneno,imgpath,tag
0,미유,쇼핑,숲속 별장처럼 꾸며진 소품 가게,제주특별자치도 제주시 한림읍 옹포리 326-3,제주특별자치도 제주시 한림읍 한림상로 15-5,33.405636,126.256762,0507-1349-9322,https://api.cdn.visitjeju.net/photomng/imgpath...,"쇼핑, 키링, 핸드메이드소품, 우산, 옹포리, 소품샵, 지갑, 한림"
1,마야블루,쇼핑,마야블루는 제주 시내의 주택가 사이에 작은 간판으로 자신의 존재를 알리고 있다. 이...,제주특별자치도 제주시 노형동 1052-27,제주특별자치도 제주시 월랑로6길 21,33.489570,126.478593,010-8515-2470,https://api.cdn.visitjeju.net/photomng/imgpath...,"라탄 소품, 악세사리, 쇼핑,라탄,원데이클래스,관광기념품,상점/상가, 쇼핑,라탄,원..."
2,몸냥공작소,쇼핑,귀엽고 제주스러운 유니크한 소품가게,제주특별자치도 제주시 애월읍 유수암리 2503-1,제주특별자치도 제주시 애월읍 하소로 595,33.430614,126.397470,--,https://api.cdn.visitjeju.net/photomng/imgpath...,"공방,기념품,쇼핑,관광기념품,상점/상가, 공방,기념품,,아주 어려움"
3,은인마켙,쇼핑,"자개, 유리 등 다양한 식기류를 만나볼 수 있는 소품샵",제주특별자치도 제주시 조천읍 함덕리 1082,제주특별자치도 제주시 조천읍 함덕서2길 20,33.540661,126.663099,010-3006-7721,https://api.cdn.visitjeju.net/photomng/imgpath...,"소품샵, 식기, 함덕, 잡화"
4,뱅뱅와인마켓,쇼핑,공항 인근에 위치한 대형 와인판매점,제주특별자치도 제주시 오라이동 2171-1,제주특별자치도 제주시 사평2길 9,33.492220,126.510559,064-746-4141,https://api.cdn.visitjeju.net/photomng/imgpath...,"와인, 오라동, 제주시내, 샴페인"


In [77]:
# data의 null갯수 확인
data.isna().sum()


title           0
category        0
introduction    0
address         0
roadaddress     0
latitude        0
longitude       0
phoneno         0
imgpath         2
tag             0
dtype: int64

> image2개가 비어있는 것 확인. 그러나 이미지는 크롤링할 때 사용할 데이터가 아니므로 상관없이 진행하기로 함.

In [ ]:
# import re
# # 문자열에서 "지하 1층", "2층" 등의 패턴을 찾아서 제거하는 함수
# def remove_floor_info(address):
#     return re.sub(r'(지하 \d+층|\d+층)', '', address)

# # 'address' 열에 있는 각 행에 대해 함수를 적용하여 패턴을 제거한 새로운 열 생성
# test_column['address_cleaned'] = test_column['address'].apply(remove_floor_info)

# # 결과 확인
# test_column['address_cleaned']

In [80]:
df = pd.DataFrame(data)

In [91]:
# 지하 1층, 1하2층 과같은 행 찾기
df_floor_address = df[df['address'].str.contains(r'(지하 \d층|\d층)', regex=True)]
df_floor_address.head()


,title,category,introduction,address,roadaddress,latitude,longitude,phoneno,imgpath,tag
8,호끄만거,쇼핑,제주에서 활동하는 작가들의 감각있는 소품을 판매하는 소품샵,제주특별자치도 제주시 일도일동 1103 지하 1층,제주특별자치도 제주시 동문로4길 9 지하 1층,33.512212,126.528202,0507-1368-1970,https://api.cdn.visitjeju.net/photomng/imgpath...,"엽서, 핸드크림, 디퓨저, 소품샵, 제주시내, 액세서리, 동문시장"
20,푸근한곰아저씨,쇼핑,올레4코스에 서 있는 쉼표 같은 책방,제주특별자치도 서귀포시 남원읍 태흥리 50-3 2층,제주특별자치도 서귀포시 남원읍 태신해안로 125 2층,33.292176,126.761385,064-764-8885,https://api.cdn.visitjeju.net/photomng/imgpath...,"책방, 소품, 잡화, 서귀포, 독립서점, 남원"
263,곶밭마켓&스튜디오,쇼핑,제주 및 전국의 다양한 전통주 판매 및 체험 클래스가 가능한 곳,제주특별자치도 서귀포시 상예동 3713-1 적벽돌 2층 건물,제주특별자치도 서귀포시 일주서로 1262 (상예동) 적벽돌 2층 건물,33.264671,126.378528,064-739-1254,https://api.cdn.visitjeju.net/photomng/imgpath...,"기념품, 전통주, 양조클래스, 전통주,공용주차장,현금결제,카드결제,화장실,무료 WI..."
513,지구별가게,쇼핑,제로웨이스트 여행을 실천할 수 있는 친환경가게,제주특별자치도 제주시 노형동 1042-2 1층,제주특별자치도 제주시 월랑로 58 1층,33.490336,126.474849,064-711-8291,https://api.cdn.visitjeju.net/photomng/imgpath...,"제로웨이스트샵,안전여행스탬프,친환경, 노형"
574,제원한의원,관광지,한국 고유의 의학인 사상체질의학을 전문으로 진료하고 있다. 사상체질을 검사하고 확진...,제주특별자치도 제주시 연동 282-52 2층,제주특별자치도 제주시 신대로14길 36 2층,33.487918,126.492857,번호없음,https://api.cdn.visitjeju.net/photomng/imgpath...,의료관광


In [92]:
# df_floor_address의 address만 보기
df_floor_address['address']

8                             제주특별자치도 제주시 일도일동 1103 지하 1층
20                           제주특별자치도 서귀포시 남원읍 태흥리 50-3 2층
263                     제주특별자치도 서귀포시 상예동 3713-1 적벽돌 2층 건물
513                             제주특별자치도 제주시 노형동 1042-2 1층
574                              제주특별자치도 제주시 연동 282-52 2층
577                           제주특별자치도 서귀포시 서홍동 397-130 2층
591                              제주특별자치도 제주시 연동 270-21 4층
669                           제주특별자치도 제주시 일도일동 1103 지하 1층
685                            제주특별자치도 제주시 용담2동 2678-2 2층
720       제주특별자치도 제주시 일도1동 1148-2 동문시장(주) 2층 윌로우씨 제주 캔들공방
977                              제주특별자치도 제주시 일도1동 1215 1층
1021                         제주특별자치도 제주시 한경면 한경해안로 156 2층
1103                           제주특별자치도 제주시 한림읍 옹포리 608 2층
1104                          제주특별자치도 제주시 이도이동 1766-18 3층
1114                      제주특별자치도 서귀포시 법환동 326-4, 2층 201호
1133                             제주특별자치도 서귀포시 하원동 1697 1층
1155                            제주특별자치도 제주시 용담일동 237-5 2층
1160          

In [94]:
# df 에서 1층, 지하2층 과같은 데이터를 지우는 함수.
df['address'] = df['address'].str.replace(r'(지하 \d층|\d층)', '', regex=True)

# 다시 df 지하 1층, 1하2층 과같은 행 찾기
df_floor_address = df[df['address'].str.contains(r'(지하 \d층|\d층)', regex=True)]
df_floor_address.head()

,title,category,introduction,address,roadaddress,latitude,longitude,phoneno,imgpath,tag


> df의 floor관련 데이터 지워진 것 확인

In [96]:
df.head()

,title,category,introduction,address,roadaddress,latitude,longitude,phoneno,imgpath,tag
0,미유,쇼핑,숲속 별장처럼 꾸며진 소품 가게,제주특별자치도 제주시 한림읍 옹포리 326-3,제주특별자치도 제주시 한림읍 한림상로 15-5,33.405636,126.256762,0507-1349-9322,https://api.cdn.visitjeju.net/photomng/imgpath...,"쇼핑, 키링, 핸드메이드소품, 우산, 옹포리, 소품샵, 지갑, 한림"
1,마야블루,쇼핑,마야블루는 제주 시내의 주택가 사이에 작은 간판으로 자신의 존재를 알리고 있다. 이...,제주특별자치도 제주시 노형동 1052-27,제주특별자치도 제주시 월랑로6길 21,33.489570,126.478593,010-8515-2470,https://api.cdn.visitjeju.net/photomng/imgpath...,"라탄 소품, 악세사리, 쇼핑,라탄,원데이클래스,관광기념품,상점/상가, 쇼핑,라탄,원..."
2,몸냥공작소,쇼핑,귀엽고 제주스러운 유니크한 소품가게,제주특별자치도 제주시 애월읍 유수암리 2503-1,제주특별자치도 제주시 애월읍 하소로 595,33.430614,126.397470,--,https://api.cdn.visitjeju.net/photomng/imgpath...,"공방,기념품,쇼핑,관광기념품,상점/상가, 공방,기념품,,아주 어려움"
3,은인마켙,쇼핑,"자개, 유리 등 다양한 식기류를 만나볼 수 있는 소품샵",제주특별자치도 제주시 조천읍 함덕리 1082,제주특별자치도 제주시 조천읍 함덕서2길 20,33.540661,126.663099,010-3006-7721,https://api.cdn.visitjeju.net/photomng/imgpath...,"소품샵, 식기, 함덕, 잡화"
4,뱅뱅와인마켓,쇼핑,공항 인근에 위치한 대형 와인판매점,제주특별자치도 제주시 오라이동 2171-1,제주특별자치도 제주시 사평2길 9,33.492220,126.510559,064-746-4141,https://api.cdn.visitjeju.net/photomng/imgpath...,"와인, 오라동, 제주시내, 샴페인"


In [100]:
# title, address만 보기
df_검색할이름 = df[['title','address']]
df_검색할이름

,title,address
0,미유,제주특별자치도 제주시 한림읍 옹포리 326-3
1,마야블루,제주특별자치도 제주시 노형동 1052-27
2,몸냥공작소,제주특별자치도 제주시 애월읍 유수암리 2503-1
3,은인마켙,제주특별자치도 제주시 조천읍 함덕리 1082
4,뱅뱅와인마켓,제주특별자치도 제주시 오라이동 2171-1
...,...,...
2566,제주돌문화공원,제주특별자치도 제주시 조천읍 교래리 산 95
2567,검멀레동굴,제주특별자치도 제주시 우도면
2568,제주러브랜드,제주특별자치도 제주시 연동 680-26
2569,한라산영실코스,제주특별자치도 서귀포시 하원동 산 1-1


> test_data의 title과 address를 합쳐서 한 컬럼으로 만들기

In [73]:
# "title"과 "address" 열을 합친 새로운 열 추가
df['title_address'] = df.apply(lambda row: f"{row['title']}, {row['address']}", axis=1)

# 결과 출력
df[['title_address']]


,title_address
0,"미유, 제주특별자치도 제주시 한림읍 옹포리 326-3"
1,"마야블루, 제주특별자치도 제주시 노형동 1052-27"
2,"몸냥공작소, 제주특별자치도 제주시 애월읍 유수암리 2503-1"
3,"은인마켙, 제주특별자치도 제주시 조천읍 함덕리 1082"
4,"뱅뱅와인마켓, 제주특별자치도 제주시 오라이동 2171-1"


> 여기까지가 title + address로 test 데이터 5개 만든 것.

> 지금부터 해야할 것 : 위의 title_address를 통해 네이버 지도에서 크롤링 가능한 범위 찾기

> 네이버지도에서 크롤링 할 목록 
- (제외)1. 사진(몇장의 사진을 들고올 것인지?, 동영상일 때 예외처리 필요) => 사진은 구글이나 네이버에서 따로 검색해서 가져오는걸로
- 2. title, title 옆 설명(분류)
- 3. 별점, 방문자 리뷰갯수, 블로그 리뷰갯수
- 4. 방문자 리뷰 최신 기준으로 정렬하여 더보기 2번 눌러서 30개 가져오기

1. title
    - 

In [7]:
# import selenium
# from selenium import webdriver
# from selenium.webdriver.common.keys import Keys

# driver = webdriver.Chrome() (윈도우용)

In [13]:
# # 크롬 드라이버 실행 (맥용)
# chrome_options = webdriver.ChromeOptions()
# driver = webdriver.Chrome(options=chrome_options)
# html = driver.page_source
# soup = BeautifulSoup(html, 'html.parser')
# count = 1


In [14]:
# Chrome Browser 와 Chrome Driver Version 확인하기
chrome_options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service = Service(ChromeDriverManager().install()),options = chrome_options)
driver.get("https://map.naver.com/v5/search/")

In [ ]:
# inputText = 